### 임베딩

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] OpenAIEmbeddings
- OpenAI API를 이용하여 텍스트를 임베딩 벡터로 변환한다.
- 모델 예시: text-embedding-3-small, text-embedding-3-large
- 가장 많이 쓰이는 기본 옵션이며, RAG 시스템 구현에 적합하다.

In [3]:
from langchain_openai import OpenAIEmbeddings

# 1) 임베딩 객체 생성
emb = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # dimensions=512   # 필요 시 차원 축소
)

# 2) 단일 질의 임베딩
query_vec = emb.embed_query("반려동물로 좋은 동물은?")
print(len(query_vec))

# 3) 여러 문서 임베딩
doc_vecs = emb.embed_documents([
    "강아지는 사람과 친숙하다.",
    "고양이는 독립적이다.",
    "자동차는 이동 수단이다."
])
print(len(doc_vecs),len(doc_vecs[0]))

1536
3 1536


In [7]:
# !pip install langchain-chroma

In [8]:
# Chroma 벡터스토어 연동 
from langchain_chroma import Chroma

texts = [
    "강아지는 사람과 친숙하다.",
    "고양이는 혼자 있는 걸 좋아한다.",
    "자동차는 이동 수단이다."
]

emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = Chroma.from_texts(texts=texts, embedding=emb)

query = "집에서 키우기 좋은 동물"
docs = vs.similarity_search(query, k=2)

for d in docs:
    print(d.page_content)

고양이는 혼자 있는 걸 좋아한다.
강아지는 사람과 친숙하다.


### [2] HuggingFaceEmbeddings
- Hugging Face Hub에서 제공하는 오픈소스 임베딩 모델을 불러와 사용.
- 예시: sentence-transformers/all-MiniLM-L6-v2, BAAI/bge-m3
- 장점: 무료, 로컬 실행 가능.
- 단점: OpenAI Embeddings 대비 정확도/속도 차이가 있을 수 있음.

In [10]:
# !pip install sentence-transformers
# !pip install hf_xet

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# 모델 선택
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 임베딩 변환
query_vec = embedding.embed_query("반려동물로 좋은 동물은?")
doc_vecs = embedding.embed_documents([
    "강아지는 사람과 친숙하다.",
    "고양이는 독립적이다.",
    "자동차는 이동 수단이다."
])

print(len(doc_vecs[0]))  # 차원 확인

384


In [14]:
# 벡터스토어 예시 (FAISS 연동)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# from transformers import logging
# logging.set_verbosity_error()

texts = [
    "강아지는 사람과 친숙하다.",
    "고양이는 혼자 있는 걸 좋아한다.",
    "자동차는 이동 수단이다."
]

embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vs = FAISS.from_texts(texts, embedding=embedding)

query = "집에서 키우기 좋은 동물"
docs = vs.similarity_search(query, k=2)

for d in docs:
    print(d.page_content)

강아지는 사람과 친숙하다.
고양이는 혼자 있는 걸 좋아한다.


### [3] CacheBackedEmbeddings
- 임베딩 계산 결과를 캐시에 저장하고, 같은 텍스트 요청이 들어오면 재계산하지 않고 캐시에서 불러옴.
- 구버전으로 Deprecated ->(현재 GitHub main branch에서도 deprecated 상태로 주석 처리되어 있음.)

In [26]:
from langchain_openai import OpenAIEmbeddings
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_classic.storage import LocalFileStore

# 기본 임베딩
base_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 캐시 저장소 지정
store = LocalFileStore("./cache_embeddings")

# 캐시 백엔드 적용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    base_embeddings,
    store,
    namespace="openai-emb"
)

# 사용
text = "강아지는 사람에게 친숙한 반려동물이다."
vec = cached_embeddings.embed_query(text)
print(" 벡터 길이:", len(vec))


C:\Users\storm\anaconda3\Lib\site-packages\langchain_classic\embeddings\cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


 벡터 길이: 1536


In [25]:
import langchain
import langchain_core
import langchain_openai

# 2025년 10월에 정식 변경
print("langchain:", langchain.__version__)              # langchain: 0.3.27  --> 1.0.1
print("langchain-core:", langchain_core.__version__)    # langchain-core: 0.3.78 --> 1.0.1
# !pip show langchain-openai                            #  langchain_openai : 1.0.1

langchain: 1.0.1
langchain-core: 1.0.1
